In [37]:
# Robustly load the module from a .py file in the current working directory to avoid
# "attempted relative import with no known parent package" when the module uses relative imports.
import sys
from pathlib import Path
import importlib.util

module_name = "Gauge_Higgs_senjanovic_HiggsDoublets"
module_path = Path.cwd() / (module_name + ".py")

if not module_path.exists():
	raise FileNotFoundError(f"Could not find {module_name}.py in the current working directory: {Path.cwd()}")

spec = importlib.util.spec_from_file_location(module_name, str(module_path))
module = importlib.util.module_from_spec(spec)

# Attempt to detect if the module lives inside a package (directory with __init__.py).
# If so, set module.__package__ and spec.name so relative imports inside the module work.
pkg_root = None
for parent in module_path.parents:
	if (parent / "__init__.py").exists():
		pkg_root = parent
		break

if pkg_root:
	pkg_parent = pkg_root.parent
	try:
		package_parts = module_path.parent.relative_to(pkg_parent).parts
	except Exception:
		package_parts = ()
	module_pkg = ".".join(package_parts) if package_parts else ""
	if module_pkg:
		spec.name = module_pkg + "." + module_name
		module.__package__ = module_pkg
	else:
		spec.name = module_name
		module.__package__ = ""
	# Ensure the package parent directory is on sys.path so sibling imports resolve.
	if str(pkg_parent) not in sys.path:
		sys.path.insert(0, str(pkg_parent))
else:
	# Not in a package: ensure the module directory is on sys.path for imports.
	if str(module_path.parent) not in sys.path:
		sys.path.insert(0, str(module_path.parent))

# register the module in sys.modules with the resolved spec.name so relative imports can find it
sys.modules[spec.name] = module
spec.loader.exec_module(module)

LHiggs_physical_approx = getattr(module, "LHiggs_physical_approx")
partial_mu = getattr(module, "partial_mu")
A = getattr(module, "A")
g = getattr(module, "g")
gBL = getattr(module, "gBL")
thw = getattr(module, "thw")
mW1 = getattr(module, "mW1")
mW2 = getattr(module, "mW2")
e = getattr(module, "e")

In [12]:
# Robustly load FeynmanRules_senjanovic_SW.py (similar approach used above for Gauge_Higgs...)
module_name = "FeynmanRules_senjanovic_SW"
module_path = Path.cwd() / (module_name + ".py")

if not module_path.exists():
	# try to find the file inside the same package folder discovered earlier (pkg_root)
	if 'pkg_root' in globals() and (pkg_root / (module_name + ".py")).exists():
		module_path = pkg_root / (module_name + ".py")
	else:
		raise FileNotFoundError(f"Could not find {module_name}.py in the current working directory: {Path.cwd()}")

fr_spec = importlib.util.spec_from_file_location(module_name, str(module_path))
fr_module = importlib.util.module_from_spec(fr_spec)

# Detect if the module lives inside a package and set package info so relative imports work.
fr_pkg_root = None
for parent in module_path.parents:
	if (parent / "__init__.py").exists():
		fr_pkg_root = parent
		break

if fr_pkg_root:
	fr_pkg_parent = fr_pkg_root.parent
	try:
		fr_package_parts = module_path.parent.relative_to(fr_pkg_parent).parts
	except Exception:
		fr_package_parts = ()
	fr_module_pkg = ".".join(fr_package_parts) if fr_package_parts else ""
	if fr_module_pkg:
		fr_spec.name = fr_module_pkg + "." + module_name
		fr_module.__package__ = fr_module_pkg
	else:
		fr_spec.name = module_name
		fr_module.__package__ = ""
	# Ensure the package parent directory is on sys.path so sibling imports resolve.
	if str(fr_pkg_parent) not in sys.path:
		sys.path.insert(0, str(fr_pkg_parent))
else:
	# Not in a package: ensure the module directory is on sys.path for imports.
	if str(module_path.parent) not in sys.path:
		sys.path.insert(0, str(module_path.parent))

# register and execute
sys.modules[fr_spec.name] = fr_module
fr_spec.loader.exec_module(fr_module)

interactions_gauge_scalars_approx = getattr(fr_module, "interactions_gauge_scalars_approx")

In [36]:
from potential_senjanovic_HiggsDoublets import HLp, HLm, GLp, GLm, HRp, HRm, GRp, GRm, k1, vR

In [13]:
interactions_gauge_scalars_approx.keys()

dict_keys([4, 3, 2, 1, 0])

In [35]:
from sympy import sin, cos, sqrt

In [43]:
ggBL_changes = {
    g**2*k1*sin(thw): 2*mW1*e,
    g*sin(thw):e,
    gBL*sqrt(cos(2*thw)):e
}

In [44]:
for interaction, coeff in interactions_gauge_scalars_approx[3].items():
    if GLm in interaction or GLp in interaction:
        if A in interaction:
            print(f"Interaction: {interaction}, Coefficient:")
            display(coeff.factor().subs(ggBL_changes))

Interaction: (A, W_1^+, {{G_L^-}}), Coefficient:


-e*m_{W_1}

Interaction: (A, W_1^-, {{G_L^+}}), Coefficient:


-e*m_{W_1}

Interaction: (A, {{G_L^+}}, {{G_L^-}}), Coefficient:


-e*(p({{G_L^+}}) - p({{G_L^-}}))

In [57]:
change_k1vR = {
    k1**2+ vR**2: vR**2
}

In [59]:
for interaction, coeff in interactions_gauge_scalars_approx[3].items():
    if GRm in interaction or GRp in interaction:
        if A in interaction:
            print(f"Interaction: {interaction}, Coefficient:")
            display(
                coeff.factor().subs(
                    ggBL_changes
                ).factor().subs(
                    change_k1vR
                ).subs(g*vR/2, mW2)
            )

Interaction: (A, W_2^+, {{G_R^-}}), Coefficient:


e*m_{W_2}

Interaction: (A, W_2^-, {{G_R^+}}), Coefficient:


e*m_{W_2}

Interaction: (A, {{G_R^+}}, {{G_R^-}}), Coefficient:


-e*(p({{G_R^+}}) - p({{G_R^-}}))

Interaction: (A, {{G_R^+}}, {{H_R^-}}), Coefficient:


0

Interaction: (A, {{G_R^-}}, {{H_R^+}}), Coefficient:


0